In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from f1_preprocessor import Preprocessor

import joblib

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor()

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [5]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [7]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting CatBoost Preprocessor (stateless)...
Fit complete.
Transforming data for CatBoost...
Applying mandatory .astype(str) to: ['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season', 'policy_report_filed_ind', 'witness_present_ind']
Dropping helper datetime columns: ['claim_date']
Transform complete.
Transforming data for CatBoost...
Applying mandatory .astype(str) to: ['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season', 'policy_report_filed_ind', 'witness_present_ind']
Dropping helper datetime columns: ['claim_date']
Transform complete.


In [8]:
print("Saving preprocessor and training columns...")

# Save the 'pre' object
joblib.dump(pre, 'cc3_preprocessor.pkl')

# Save the exact column order and names
joblib.dump(X_train_proc.columns, 'training_columns.pkl')

print("Done.")

Saving preprocessor and training columns...
Done.


## CatBoost with Optuna Tuning

In [9]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
CAT_FEATURES = pre.cat_features_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season', 'policy_report_filed_ind', 'witness_present_ind']


In [11]:
def objective(trial: optuna.trial.Trial) -> float:

    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True), 
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'early_stopping_rounds': 100,
        'random_state': 123
    }

    params['eval_metric'] = 'Logloss'
    
    model = CatBoostClassifier(**params)
    
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_test_proc, y_test),
        cat_features=CAT_FEATURES,
        verbose=False
    )

    y_preds = model.predict(X_test_proc)
    
    manual_f1_score = f1_score(y_test, y_preds, pos_label=1)
    
    return manual_f1_score

In [12]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=100, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-10 12:49:08,895] A new study created in memory with name: no-name-127f4cdf-286c-40e8-83e3-a85bd5322b00



2. Starting Optuna study...


Best trial: 0. Best value: 0.541975:   1%|          | 1/100 [00:03<05:19,  3.23s/it]

[I 2025-11-10 12:49:12,132] Trial 0 finished with value: 0.5419749936208217 and parameters: {'learning_rate': 0.05107592729130285, 'depth': 9, 'l2_leaf_reg': 0.0012021005496685258, 'subsample': 0.7, 'random_strength': 3.963946965037201e-05, 'bagging_temperature': 0.844561474638459, 'border_count': 214, 'scale_pos_weight': 7.262763324169601}. Best is trial 0 with value: 0.5419749936208217.


Best trial: 0. Best value: 0.541975:   2%|▏         | 2/100 [00:07<06:04,  3.71s/it]

...

[I 2025-11-10 12:55:32,063] Trial 97 finished with value: 0.5976391231028668 and parameters: {'learning_rate': 0.018022536068975113, 'depth': 7, 'l2_leaf_reg': 2.4961654148918226, 'subsample': 0.7, 'random_strength': 1.2640078144978489e-05, 'bagging_temperature': 0.05558101201670897, 'border_count': 116, 'scale_pos_weight': 2.524851995252126}. Best is trial 39 with value: 0.6021872863978127.


Best trial: 39. Best value: 0.602187:  99%|█████████▉| 99/100 [06:31<00:05,  5.55s/it]

[I 2025-11-10 12:55:40,231] Trial 98 finished with value: 0.5945945945945946 and parameters: {'learning_rate': 0.011722651131210722, 'depth': 8, 'l2_leaf_reg': 0.3035834498158361, 'subsample': 0.7, 'random_strength': 3.0244426454653993e-05, 'bagging_temperature': 0.06724048934367913, 'border_count': 137, 'scale_pos_weight': 2.528068824282113}. Best is trial 39 with value: 0.6021872863978127.


Best trial: 39. Best value: 0.602187: 100%|██████████| 100/100 [06:37<00:00,  3.98s/it]

[I 2025-11-10 12:55:46,766] Trial 99 finished with value: 0.5860148514851485 and parameters: {'learning_rate': 0.01056762156000304, 'depth': 9, 'l2_leaf_reg': 0.11116318860041133, 'subsample': 0.8, 'random_strength': 1.397799542205658e-05, 'bagging_temperature': 0.27360809514225526, 'border_count': 116, 'scale_pos_weight': 3.4583098585874525}. Best is trial 39 with value: 0.6021872863978127.

Optuna study finished.
Number of finished trials: 100

Best trial:
  Value (Max F1 Score): 0.6022
  Best Hyperparameters:
    learning_rate: 0.042059298117722826
    depth: 6
    l2_leaf_reg: 2.6676457775670617
    subsample: 0.5
    random_strength: 0.07041736275037083
    bagging_temperature: 0.8299484798553904
    border_count: 35
    scale_pos_weight: 2.3543968662300707


In [13]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 2000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 100, # Keep early stopping
    'random_state': 123
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.042059298117722826, 'depth': 6, 'l2_leaf_reg': 2.6676457775670617, 'subsample': 0.5, 'random_strength': 0.07041736275037083, 'bagging_temperature': 0.8299484798553904, 'border_count': 35, 'scale_pos_weight': 2.3543968662300707}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.6022

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.90      0.81      0.85      4165
   Class 1.0       0.52      0.71      0.60      1235

    accuracy                           0.78      5400
   macro avg       0.71      0.76      0.73      5400
weighted avg       0.82      0.78      0.79      5400



In [14]:
print("Saving best_model...")

# Use the model's F1 score in the name
best_model.save_model("catboost_mod_f1_0.6022.cbm")

print("Done.")

Saving best_model...
Done.


In [15]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 83

Top 10 Most Important Features:
                     feature  importance
14                 liab_prct   23.422263
49     recovery_case_clarity    6.895557
52      highrisk_site_binary    3.970960
46             is_single_car    3.845630
13       witness_present_ind    2.992513
45  is_multi_vehicle_unclear    2.951461
25       in_network_bodyshop    2.888566
53            high_risk_site    2.612414
48       has_recovery_target    2.430656
22            vehicle_weight    2.138397

Bottom 10 Least Important Features:
              feature  importance
66     veteran_driver    0.052804
43      police_binary    0.047522
62  middle_age_driver    0.035613
36          is_friday    0.028452
59       young_driver    0.023122
38         claim_hour    0.000000
64      novice_driver    0.000000
60      senior_driver    0.000000
39          rush_hour    0.000000
40         late_night    0.000000

Keeping top 66 features and removing bottom 100.


In [16]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data for CatBoost...
Applying mandatory .astype(str) to: ['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season', 'policy_report_filed_ind', 'witness_present_ind']
Dropping helper datetime columns: ['claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            0
2       4655051            0
3       6728725            1
4       9848460            1


In [17]:
prediction.to_csv("results/catboost_6022_prediction.csv", index=False)